<a href="https://colab.research.google.com/github/GabrielFurnielesGarcia/deeplearning-2024/blob/main/ModelMerging/LABFinal_ModelMerging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Merging

Sofía Ángel, Gabriel Furnieles

Mayo 2024

---

>[Model Merging](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=m0AYB3Ed75uT)

>>[¿Qué es Model Merging?](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=l7zubhCd8_Q4)

>>[Model Merging techniques](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=BtONg7j7CdAz)

>>>[SLERP](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=2O1BQ3c2CirM)

>>>[TIES](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=99kxfMkVCk08)

>>>[DARE](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=e_pyTa1GCzgj)



## ¿Qué es Model Merging?

## Model Merging techniques

### SLERP

### TIES

### DARE

### Passthrough

## Utilizando la librería `merge-kit`

In [ ]:
MODEL_NAME = "NeuralPipe-7B-slerp"
yaml_config = """
slices:
  - sources:
      - model: OpenPipe/mistral-ft-optimized-1218
        layer_range: [0, 32]
      - model: mlabonne/NeuralHermes-2.5-Mistral-7B
        layer_range: [0, 32]
merge_method: slerp
base_model: OpenPipe/mistral-ft-optimized-1218
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
"""

## Evaluando

## Evolutionary Model Merging

## Referencias